# __Code__

베이스라인 코드에서 크게 수정하지는 않았습니다.  
EfficientNet-B7 모델 2개로 앙상블을 수행하였고,  
**실제 학습을 수행하였을 때는 세션 여러개를 띄워 각 fold를 할당하여 학습하였습니다.**

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# __Data__

In [ ]:
from google.colab import output

!cp "/content/drive/MyDrive/Dacon_Emnist2/data_2.zip" "data_2.zip"
!unzip "data_2.zip"

In [ ]:
!mkdir "./dirty_mnist"
!unzip "dirty_mnist_2nd.zip" -d "./dirty_mnist/"
!mkdir "./test_dirty_mnist"
!unzip "test_dirty_mnist_2nd.zip" -d "./test_dirty_mnist/"
output.clear()

# __Library Import & Download Efficentnet_pytorch__

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import imutils
import zipfile
import os
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from efficientnet_pytorch import EfficientNet
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
dirty_mnist_answer = pd.read_csv("dirty_mnist_2nd_answer.csv")
dirty_mnist_answer

# __Data Preprocessing__

RandomFlip을 비롯한 다양한 Augmentation을 실험하였지만  
오히려 성능이 하락하였기 때문에  
간단하게 RandomRoation만 적용하였습니다.

In [ ]:
namelist = os.listdir('./dirty_mnist/')

# numpy를 tensor로 변환하는 ToTensor 정의
class ToTensor(object):
    """numpy array를 tensor(torch)로 변환합니다."""
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.FloatTensor(image),
                'label': torch.FloatTensor(label)}
# to_tensor 선언

to_tensor = T.Compose([
                      ToTensor()
])

# Augmentation은 horizontal, vertical flip도 넣고 실험해봤지만
# 성능이 오히려 떨어짐 -> RandomRotation만 사용
augmentations = T.Compose([
                           T.ToPILImage(),
                           T.RandomRotation(40),
                           T.ToTensor()
                           
])

class DatasetMNIST(torch.utils.data.Dataset):
    def __init__(self,
                 dir_path,
                 meta_df,
                 transforms=to_tensor,#미리 선언한 to_tensor를 transforms로 받음
                 augmentations=None):
        
        self.dir_path = dir_path # 데이터의 이미지가 저장된 디렉터리 경로
        self.meta_df = meta_df # 데이터의 인덱스와 정답지가 들어있는 DataFrame

        self.transforms = transforms# Transform
        self.augmentations = augmentations # Augmentation
        
    def __len__(self):
        return len(self.meta_df)
    
    def __getitem__(self, index):
        # 폴더 경로 + 이미지 이름 + .png => 파일의 경로
        # 참고) "12".zfill(5) => 000012
        #       "146".zfill(5) => 000145
        # cv2.IMREAD_GRAYSCALE : png파일을 채널이 1개인 GRAYSCALE로 읽음
        image = cv2.imread(self.dir_path +\
                           str(self.meta_df.iloc[index,0]).zfill(5) + '.png',
                           cv2.IMREAD_GRAYSCALE)
        # 0 ~ 255의 값을 갖고 크기가 (256,256)인 numpy array를
        # 0 ~ 1 사이의 실수를 갖고 크기가 (256,256,1)인 numpy array로 변환
        image = (image/255).astype('float32')[..., np.newaxis]

        # 정답 numpy array생성(존재하면 1 없으면 0)
        label = self.meta_df.iloc[index, 1:].values.astype('float')
        sample = {'image': image, 'label': label}

        # transform 적용
        # numpy to tensor
        if self.transforms:
            sample = self.transforms(sample)
        if self.augmentations:
            sample['image'] = self.augmentations(sample['image'])
        # sample 반환
        return sample

# __Model__

두명의 팀원 모두 EfficientNet-B7으로 진행.  
두 모델은 출력 이전 층에서 dropout의 적용 여부 및 activation 함수의 종류만 다르고  
나머지는 동일합니다.

In [ ]:
################# H.J.S Model #################

class MultiLabelEfficientnet(nn.Module):
    def __init__(self):
        super(MultiLabelEfficientnet, self).__init__()
        self.conv2d = nn.Conv2d(1, 3, 3, stride=1)
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b7')
        self.drop = nn.Dropout(p=0.2)
        self.FC = nn.Linear(1000, 26)

    def forward(self, x):
       
        x = F.silu(self.conv2d(x))

        # efficientnet b7
        # 원래는 efficientnet과 동일하게 swish를 넣으려고 했으나
        # 별 차이 없는 듯 해서 그냥 silu 적용
        x = F.silu(self.efficientnet(x))

        # 마지막 출력에 nn.Linear를 추가
        # multilabel을 예측해야 하기 때문에
        # softmax가 아닌 sigmoid를 적용
        # dropout 추가
        x = self.drop(x)
        x = torch.sigmoid(self.FC(x))
        return x
# 모델 선언
model = MultiLabelEfficientnet()
model

In [ ]:
################# 잉돌 Model #################

from efficientnet_pytorch import EfficientNet
class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

# A memory-efficient implementation of Swish function
class SwishImplementation(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_tensors[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))

class MemoryEfficientSwish(nn.Module):
    def forward(self, x):
        return SwishImplementation.apply(x)
        
class MultiLabelEfficientnet2(nn.Module):
    def __init__(self):
        super(MultiLabelEfficientnet2, self).__init__()
        self.conv2d = nn.Conv2d(1, 3, 3, stride=1)
        self._swish = MemoryEfficientSwish()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b7')
        self.fc = nn.Linear(1000, 26)

    def forward(self, x):
        # 3개의 채널(color)을 갖기 때문에
        # 입력 전에 conv2d를 한 층 추가
        # swish 사용
        x = self._swish(self.conv2d(x))
        # effnet 추가
        x = self._swish(self.efficientnet(x))
        # 마지막 출력에 nn.Linear를 추가
        # multilabel을 예측해야 하기 때문에
        # softmax가 아닌 sigmoid 적용(0~1)
        x = torch.sigmoid(self.fc(x))
        return x


# 모델 선언
model2 = MultiLabelEfficientnet2()
# model2.eval()
model2

# __Train__

실제 학습은 두 명이 각자 수행하였고,  
베이스라인 코드와 마찬가지로 5 fold로 진행하였습니다.  
아래 코드와는 다르게 실제 훈련 시에는 세션 여러개를 띄워  
각 fold를 할당하여 진행하였습니다.

In [ ]:
##### MultiLabelEfficientnet Train #####

from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)



# dirty_mnist_answer에서 train_idx와 val_idx를 생성
best_models = [] # 폴드별로 가장 validation acc가 높은 모델 저장
for fold_index, (trn_idx, val_idx) in enumerate(kfold.split(dirty_mnist_answer),1):
    
    #########################################

    # 다섯 폴드를 코랩에서 그대로 다 돌리는 것은 불가능해서
    # 사본 여러개 띄우고 폴드 할당해서 훈련 + 저장
    # evaluate은 저장한 폴드별 best model을 불러와서 사용

    # 1
    # if fold_index > 1:
    #   break
    
    # 2
    # if fold_index == 1:
    #   continue
    # elif fold_index > 2:
    #   break

    # 3
    # if fold_index < 3:
    #   continue
    # elif fold_index > 3:
    #   break

    #4
    # if fold_index < 4:
    #   continue
    # elif fold_index > 4:
    #   break

    # 5
    # if fold_index != 5:
    #   break

    #########################################
    
    print(f'[fold: {fold_index}]')
    
    # cuda cache 초기화
    torch.cuda.empty_cache()

    #train fold, validation fold 분할
    train_answer = dirty_mnist_answer.iloc[trn_idx]
    test_answer  = dirty_mnist_answer.iloc[val_idx]

    #Dataset 정의
    train_dataset = DatasetMNIST("dirty_mnist/", train_answer, augmentations=augmentations)
    valid_dataset = DatasetMNIST("dirty_mnist/", test_answer)

    #DataLoader 정의
    train_data_loader = DataLoader(
        train_dataset,
        batch_size = 16,
        shuffle = True,
        num_workers = 3
    )
    valid_data_loader = DataLoader(
        valid_dataset,
        batch_size = 8,
        shuffle = False,
        num_workers = 3
    )

    # 모델 선언
    model = MultiLabelEfficientnet()
    model.to(device)# gpu에 모델 할당

    # 훈련 옵션 설정
    optimizer = torch.optim.Adam(model.parameters(),
                                lr = 0.001)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size = 5,
                                                gamma = 0.9)
    criterion = torch.nn.BCELoss()
    # 훈련 시작
    valid_acc_max = 0
    valid_loss_min = float("inf")
    for epoch in range(30):
        # 1개 epoch 훈련
        train_acc_list = []
        train_loss_list = []
        with tqdm(train_data_loader,#train_data_loader를 iterative하게 반환
                total=train_data_loader.__len__(), # train_data_loader의 크기
                unit="batch") as train_bar:# 한번 반환하는 sample의 단위는 "batch"
            for sample in train_bar:
                train_bar.set_description(f"Train Epoch {epoch}")
               
                optimizer.zero_grad()
                images, labels = sample['image'], sample['label']
                # tensor를 gpu에 올리기 
                images = images.to(device)
                labels = labels.to(device)

                # 모델의 dropoupt, batchnormalization를 train 모드로 설정
                model.train()
                # .forward()에서 중간 노드의 gradient를 계산
                with torch.set_grad_enabled(True):
                    # 모델 예측
                    probs  = model(images)
                    # loss 계산
                    loss = criterion(probs, labels)
                    # 중간 노드의 gradient로
                    # backpropagation을 적용하여
                    # gradient 계산
                    loss.backward()
                    # weight 갱신
                    optimizer.step()

                    # train accuracy 계산
                    probs  = probs.cpu().detach().numpy()
                    labels = labels.cpu().detach().numpy()
                    preds = probs > 0.5
                    batch_acc = (labels == preds).mean()    
                    train_acc_list.append(batch_acc)
                    train_acc = np.mean(train_acc_list)
                    train_loss_list.append(loss.item())
                    train_loss = np.mean(train_loss_list)
                # 현재 progress bar에 현재 미니배치의 loss 결과 출력
                train_bar.set_postfix(train_loss= train_loss,
                                      train_acc = train_acc)
                

        # 1개 epoch학습 후 Validation 점수 계산
        valid_acc_list = []
        valid_loss_list = []
        with tqdm(valid_data_loader,
                total=valid_data_loader.__len__(),
                unit="batch") as valid_bar:
            for sample in valid_bar:
                valid_bar.set_description(f"Valid Epoch {epoch}")
                optimizer.zero_grad()
                images, labels = sample['image'], sample['label']
                images = images.to(device)
                labels = labels.to(device)

                # 모델의 dropoupt, batchnormalization를 eval모드로 설정
                model.eval()
                # .forward()에서 중간 노드의 gradient를 계산
                with torch.no_grad():
                    # validation loss만을 계산
                    probs  = model(images)
                    valid_loss = criterion(probs, labels)

                    # train accuracy 계산
                    probs  = probs.cpu().detach().numpy()
                    labels = labels.cpu().detach().numpy()
                    preds = probs > 0.5
                    batch_acc = (labels == preds).mean()
                    valid_acc_list.append(batch_acc)
                    valid_loss_list.append(valid_loss.item())
                valid_acc = np.mean(valid_acc_list)
                valid_loss = np.mean(valid_loss_list)
                valid_bar.set_postfix(valid_loss = valid_loss,
                                      valid_acc = valid_acc)
            
        # Learning rate 조절
        lr_scheduler.step()

        # 모델 저장
        if valid_loss_min > valid_loss:
            valid_loss_min = valid_loss
            best_model = model
            
            

    # 폴드별로 가장 좋은 모델 저장
    MODEL = "efficientnetb7"
    # 모델을 저장할 구글 드라이브 경로
    path = "/content/drive/MyDrive/Dacon_Emnist2/"
    torch.save(best_model, f'{path}{MODEL}_{fold_index}_{valid_loss:2.4f}.pth')
    best_models.append(best_model)

In [ ]:
##### MultiLabelEfficientnet2 Train #####

# cross validation을 적용하기 위해 KFold 생성
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# dirty_mnist_answer에서 train_idx와 val_idx를 생성
best_models2 = [] # 폴드별로 가장 validation acc가 높은 모델 저장
for fold_index, (trn_idx, val_idx) in enumerate(kfold.split(dirty_mnist_answer),1):

    # if fold_index > 1:
    #   break

    print(f'[fold: {fold_index}]')
    # cuda cache 초기화
    torch.cuda.empty_cache()

    #train fold, validation fold 분할
    train_answer = dirty_mnist_answer.iloc[trn_idx]
    valid_answer  = dirty_mnist_answer.iloc[val_idx]

    #Dataset 정의
    train_dataset = DatasetMNIST("dirty_mnist/", train_answer, augmentations=augmentations) # use augmentations param at trainset
    valid_dataset = DatasetMNIST("dirty_mnist/", valid_answer)

    #DataLoader 정의
    train_data_loader = DataLoader(
        train_dataset,
        batch_size = 16,
        shuffle = False,
        num_workers = 3
    )
    valid_data_loader = DataLoader(
        valid_dataset,
        batch_size = 8,
        shuffle = False,
        num_workers = 3
    )

    model2 = MultiLabelEfficientnet2() #모델 선언
    model2.to(device)# gpu에 모델 할당

    # 훈련 옵션 설정
    optimizer = torch.optim.Adam(model2.parameters(),
                                lr = 0.001)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                step_size = 5, #for each 5 epochs
                                                gamma = 0.85) #current_lr*0.85
    #binary cross entropy
    criterion = torch.nn.BCELoss()

    # 훈련 시작
    valid_acc_max = 0
    for epoch in range(30): #30 epoch
        # 1개 epoch 훈련
        train_acc_list = []
        train_loss_list = []
        with tqdm(train_data_loader,#train_data_loader를 iterative하게 반환
                  total=train_data_loader.__len__(), # train_data_loader의 크기
                  unit="batch"
                  ) as train_bar:# 한번 반환하는 sample의 단위는 "batch"
            for sample in train_bar:
                train_bar.set_description(f"Train Epoch {epoch}")
                # 갱신할 변수들에 대한 모든 변화도를 0으로 초기화
                # 참고)https://tutorials.pytorch.kr/beginner/pytorch_with_examples.html
                optimizer.zero_grad()
                images, labels = sample['image'], sample['label']
                # tensor를 gpu에 올리기 
                images = images.to(device)
                labels = labels.to(device)

                # 모델의 dropoupt, batchnormalization를 train 모드로 설정
                model2.train()
                # .forward()에서 중간 노드의 gradient를 계산
                with torch.set_grad_enabled(True):
                    # 모델 예측
                    probs  = model2(images)
                    # loss 계산
                    loss = criterion(probs, labels)
                    # 중간 노드의 gradient로
                    # backpropagation을 적용하여
                    # gradient 계산
                    loss.backward()
                    # weight 갱신
                    optimizer.step()
                    # train accuracy 및 loss 계산
                    probs  = probs.cpu().detach().numpy()
                    labels = labels.cpu().detach().numpy()
                    preds = probs > 0.5
                    batch_acc = (labels == preds).mean()    
                    train_acc_list.append(batch_acc)
                    train_acc = np.mean(train_acc_list)
                    train_loss_list.append(loss.item())
                    train_loss = np.mean(train_loss_list)
                # 현재 progress bar에 현재 미니배치의 loss 결과 출력
                train_bar.set_postfix(train_loss= train_loss,
                                      train_acc = train_acc)
                

        # 1개 epoch학습 후 Validation 점수 계산
        valid_acc_list = []
        valid_loss_list = []
        with tqdm(valid_data_loader,
                total=valid_data_loader.__len__(),
                unit="batch") as valid_bar:
            for sample in valid_bar:
                valid_bar.set_description(f"Valid Epoch {epoch}")
                optimizer.zero_grad()
                images, labels = sample['image'], sample['label']
                images = images.to(device)
                labels = labels.to(device)

                # 모델의 dropoupt, batchnormalization를 eval모드로 설정
                model2.eval()
                # .forward()에서 중간 노드의 gradient를 계산
                with torch.no_grad():
                    # acc, validation loss를 계산
                    probs  = model2(images)
                    valid_loss = criterion(probs, labels)

                    # train accuracy 계산
                    probs  = probs.cpu().detach().numpy()
                    labels = labels.cpu().detach().numpy()
                    preds = probs > 0.5
                    batch_acc = (labels == preds).mean()
                    valid_acc_list.append(batch_acc)
                    valid_loss_list.append(valid_loss.item())
                valid_acc = np.mean(valid_acc_list)
                valid_loss = np.mean(valid_loss_list)
                valid_bar.set_postfix(valid_loss = valid_loss,
                                      valid_acc = valid_acc)
            
        # Learning rate 조절
        lr_scheduler.step()

        # 모델 저장
        if valid_acc_max < valid_acc:
            valid_acc_max = valid_acc
            best_model = model2
            MODEL = "Effnetb7_3th"
            # 모델을 저장할 구글 드라이브 경로
            path = "/content/drive/MyDrive/Dacon_Emnist2/"
            torch.save(best_model, f'{path}{fold_index}_{MODEL}_{valid_loss.item():2.4f}_epoch_{epoch}.pth')

    # 폴드별로 가장 좋은 모델 저장
    best_models2.append(best_model)

In [ ]:
cd /content/.

# __Inference__

In [ ]:
# 폴드별 best model 드라이브에서 로드하기
# best_models = []
# best_models.append(torch.load('./drive/MyDrive/Dacon_Emnist2/efficientnetb7_1_0.1780.pth'))
# best_models.append(torch.load('./drive/MyDrive/Dacon_Emnist2/2_efficientnetb7_0.1672.pth'))
# best_models.append(torch.load('./drive/MyDrive/Dacon_Emnist2/3_efficientnetb7_0.1581.pth'))
# best_models.append(torch.load('./drive/MyDrive/Dacon_Emnist2/efficientnetb7_4_0.1610.pth'))
# best_models.append(torch.load('./drive/MyDrive/Dacon_Emnist2/efficientnetb7_5_0.1645.pth'))

In [ ]:
sample_submission = pd.read_csv("sample_submission.csv")
test_dataset = DatasetMNIST("test_dirty_mnist/", sample_submission)
batch_size = 32
test_data_loader = DataLoader(
    test_dataset,
    batch_size = batch_size,
    shuffle = False,
    num_workers = 3,
    drop_last = False
)
test_data_loader2 = DataLoader(
    test_dataset,
    batch_size = 128,
    shuffle = False,
    num_workers = 3,
    drop_last = False
)

In [ ]:
predictions_list = []
# 배치 단위로 추론
prediction_df = pd.read_csv("sample_submission.csv")

# 5개의 fold마다 가장 좋은 모델을 이용하여 예측
for model in best_models:
    # 0으로 채워진 array 생성
    prediction_array = np.zeros([prediction_df.shape[0],
                                 prediction_df.shape[1] -1])
    for idx, sample in enumerate(test_data_loader):
        with torch.no_grad():
            # 추론
            model.eval()
            images = sample['image']
            images = images.to(device)
            probs  = model(images)
            probs = probs.cpu().detach().numpy()
            preds = (probs > 0.5)

            # 예측 결과를 
            # prediction_array에 입력
            batch_index = batch_size * idx
            prediction_array[batch_index: batch_index + images.shape[0],:]\
                         = preds.astype(int)
                         
    # 채널을 하나 추가하여 list에 append
    predictions_list.append(prediction_array[...,np.newaxis])

In [ ]:
predictions_list2 = []
for model in best_models2:
    # 0으로 채워진 array 생성
    prediction_array2 = np.zeros([prediction_df.shape[0],
                                 prediction_df.shape[1] -1])
    for idx, sample in enumerate(test_data_loader2):
        with torch.no_grad():
            # 추론
            model.eval()
            images = sample['image']
            images = images.to(device)
            probs  = model(images)
            probs = probs.cpu().detach().numpy()
            preds = (probs > 0.5)                

            # 예측 결과를 
            # prediction_array에 입력
            batch_index = batch_size * idx
            prediction_array2[batch_index: batch_index + images.shape[0],:]\
                         = preds.astype(int)
                         
    # 채널을 하나 추가하여 list에 append
    predictions_list2.append(prediction_array[...,np.newaxis])

In [ ]:
# 모델 전체를 불러오는 것이 아니라 확률값을 불러와서 적용

# prediction_array_song = np.load('./drive/MyDrive/Dacon_Emnist2/predictions_b7_3th_fold5_song.dat',allow_pickle=True)
# predictions_array = np.concatenate(predictions_list, axis = 2)
# predictions_array_final = np.concatenate([predictions_array, prediction_array_song], axis=2)

# __Ensemble__

In [ ]:
########## Ensemble Code ##########
predictions_array1 = np.concatenate(predictions_list, axis=2)
predictions_array2 = np.concatenate(predictions_list2, axis=2)
predictions_array_final = np.concatenate([predictions_array1, predictions_array2], axis=2)
predictions_mean = predictions_array_final.mean(axis = 2)
# 평균 값이 0.5보다 클 경우 1 작으면 0
predictions_mean = (predictions_mean > 0.5) * 1

In [ ]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission.iloc[:,1:] = predictions_mean

In [ ]:
cd /content/drive/MyDrive/

In [ ]:
sample_submission.to_csv("./Dacon_Emnist2/final_test_prediction.csv", index = False)
sample_submission